In [12]:
rm -rf obitools4
git clone https://github.com/metabarcoding/obitools4 

Cloning into 'obitools4'...
remote: Enumerating objects: 7574, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 7574 (delta 287), reused 384 (delta 148), pack-reused 6983 (from 2)
Receiving objects: 100% (7574/7574), 188.68 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (4721/4721), done.
Updating files: 100% (529/529), done.


In [13]:
### Fixer la version 
cd obitools4
git reset --hard 996ec69
cd ..

HEAD is now at 996ec69 update the release notes for version 4.4.0


In [14]:
conda install -c conda-forge compilers=1.10 make=4.4.1 go=1.24.5 zlib=1.3.1 jq=1.8.1 --yes

Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [15]:
# CHANGER LE LIEN DU GITLAB EN GITHUB
find . -type f -exec sed -i \
  -e 's|git\.metabarcoding\.org/obitools/obitools4/obitools4|github.com/metabarcoding/obitools4|g' \
  -e 's|git\.metabarcoding\.org/org-asm/org-asm\.git|github.com/metabarcoding/org-asm.git|g' \
  -e 's|git\.metabarcoding\.org/lecasofts/go/obitools|github.com/metabarcoding/obitools4|g' \
  -e 's|https://git\.metabarcoding\.org|https://github.com/metabarcoding|g' \
  {} +


In [16]:
export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-cc
export CGO_ENABLED=1
export CGO_CFLAGS="-I$CONDA_PREFIX/include"
export CGO_LDFLAGS="-L$CONDA_PREFIX/lib"

In [ ]:
# Installation d'obitools V4
cd obitools4
make
cd ..

In [30]:
# Recherche des chemins des inputs et formatage de l'ouput 
json_path="galaxy_inputs/galaxy_inputs.json"

# input_file="galaxy_inputs/wolfdiet_obigrep.fastq" test injection d'input
input_file=$(grep -o '"path": *"[^"]*"' "$json_path" | head -n 1 | sed 's/.*"path": *"\([^"]*\)".*/\1/')

echo "Input file path: $input_file"


filename=$(basename -- "$input_file")       # ex: myfile.fastq.gz
extension="${filename##*.}"                # ex: gz
filename="${filename%.*}"                  # ex: myfile.fastq ou myfile

echo "Nom de fichier (sans chemin, sans extension) : $filename"
echo "Extension détectée : $extension"


output_file="outputs/outputfile.$extension"
format=""


Input file path: galaxy_inputs/wolfdiet_obigrep.fastq
Nom de fichier (sans chemin, sans extension) : wolfdiet_obigrep
Extension détectée : fastq


In [ ]:
### Create directory output 

savegraph=$(jq -r '.savegraph' galaxy_inputs/galaxy_inputs.json)
saveratio=$(jq -r '.saveratio' galaxy_inputs/galaxy_inputs.json)
if [[ "$savegraph" == "True" || "$saveratio" == "True" ]]
    mkdir outputs/save
    save_dir="outputs/save"
    if [["$savegraph" == "True"]]
        savegraph="--save-graph $save_dir"
    else
        savegraph=""
    fi
    if [["$saveratio" == "True"]]
        saveratio="--save-ratio $save_dir"
    else
        saveratio=""
    fi
fi

minievalrate=$(jq -r '.minievalrate' galaxy_inputs/galaxy_inputs.json)

if [[ -n "$minevalrate" && -n "$saveratio" ]]; then
    minevalrate="--min-eval-rate $minevalrate"
fi

In [ ]:
#### Variables
# distance="" INTEGER -d
dist=$(jq -r '.dist' galaxy_inputs/galaxy_inputs.json)
if [[ -n "$dist"  && "$dist" != "null" ]]; then
    dist="-d $dist"
    echo "variable distance présente: $dist"
else 
    dist=""
fi


# ratio=""  FLOAT -r 
ratio=$(jq -r '.ratio' galaxy_inputs/galaxy_inputs.json)
if [[ -n "$ratio" && "$ratio" != "null" ]]; then
    ratio="-r $ratio"
    echo "variable ratio présente: $ratio"
else 
    ratio=""            
fi

# sample="" STRING -s
sample=$(jq -r '.sample' galaxy_inputs/galaxy_inputs.json)
if [[ -n "$sample" && "$sample" != "null" ]]; then
    sample="-s $sample"  
    echo "variable sample présente: $sample"
else 
    sample=""
fi

# chimera BOOLEEN --detect-chimera
chimera=$(jq -r '.chimera' galaxy_inputs/galaxy_inputs.json)
if [[ "$chimera" == "true" ]]; then
    chimera="--detect-chimera"
    echo "variable chimera présente: $chimera"
else 
    chimera=""
fi

# head BOOLEEN -H
head=$(jq -r '.head' galaxy_inputs/galaxy_inputs.json)
if [[ "$head" == "true" ]]; then
    head="-H"
    echo "variable head présente: $head"
else
    head=""
fi  

# min sample count INTEGER --min-sample-count
mincount=$(jq -r '.mincount' galaxy_inputs/galaxy_inputs.json)
if [[ -n "$mincount" && "$mincount" != "null" ]]; then
    mincount="--min-sample-count $mincount"
    echo "variable mincount présente: $mincount"
else 
    mincount=""
fi      


####


seq: 
def: 
ident: 
attrib: 
hasattrib: 
predicat: 
bash: jq: command not found
bash: jq: command not found




In [ ]:
./obitools4/build/obiclean $dist $ratio $sample $chimera $head $mincount $savegraph $saveratio $minievalrate $input_file --out $output_file

INFO[0000] Number of workers set 125                    
INFO[0000] Found 1 files to process                     
INFO[0000] galaxy_inputs/wolfdiet_obigrep.fastq mime type: text/fastq 
INFO[0000] On output use JSON headers                   
| Reading sequences (41435/-, 9924 it/s) [4s]  
